In [ ]:
from progressivis import Scheduler
from progressivis.io import CSVLoader
from progressivis.stats import Histogram2D, Min, Max
from progressivis.datasets import get_dataset
from progressivis.vis import Heatmap
s = Scheduler.default = Scheduler()
URLS = [f"https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2015-0{n}.csv" for n in range(1,7)]
csv_module = CSVLoader(URLS, index_col=False, skipinitialspace=True,
                usecols=['pickup_longitude', 'pickup_latitude']) # load many compressed CSV files
min_module = Min() # computes the min value of each column
min_module.input.table = csv_module.output.result
max_module = Max() # computes the max value of each column
max_module.input.table = csv_module.output.result
histogram2d = Histogram2D('pickup_longitude', # compute a 2d histogram
                          'pickup_latitude', 
                          xbins=256, ybins=256)
histogram2d.input.table = csv_module.output.result
histogram2d.input.min = min_module.output.result
histogram2d.input.max = max_module.output.result
heatmap=Heatmap(filename='histo_tmp.png') # compute the Heatmap
heatmap.input.array = histogram2d.output.result

In [ ]:
import ipywidgets as ipw
from IPython.display import display
wg = None
async def _after_run(m, run_number):
    global wg
    if m.result is None or not len(m.result):
        return
    filename = m.result.last()['filename']
    img = open(filename, 'rb').read()
    if wg is None:
        wg = ipw.Image(value=img)
        display(wg)
    else:
        wg.value = img
    
heatmap.after_run_proc = _after_run

await s.start()